In [ ]:
from PIL import Image
import os
import torch
import numpy as np

from utils import get_device_map

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
devices = [0, 4, 5]
start_device = 'cuda:' + str(devices[0])

In [ ]:
checkpoint = "Salesforce/blip2-opt-2.7b"
result_save_path = '../results/coco_val_blip2.csv'
# cache_dir = "/mnt/nas2/kjh/huggingface_cache"
cache_dir = "../caches"
cache_pretrained_files_dir = os.path.join(cache_dir, "pretrained_files")
cache_dataset_dir = os.path.join(cache_dir, "datasets")
dtype = torch.float16
batch_size = 16
num_workers = 8
max_new_tokens = 50

### Processor

In [ ]:
from transformers import Blip2Processor

processor = Blip2Processor.from_pretrained(
    checkpoint,
    cache_dir=cache_pretrained_files_dir,
)


### Model

In [ ]:
from transformers import Blip2ForConditionalGeneration

device_map = get_device_map(checkpoint, devices)

model = Blip2ForConditionalGeneration.from_pretrained(
    checkpoint,
    cache_dir=cache_pretrained_files_dir,
    torch_dtype=dtype,
    # device_map='auto',
    device_map=device_map
)

In [ ]:
# Freeze
freeze_list = [
    model.vision_model,
    model.qformer,
    model.language_projection,
    model.language_model,
]

for freeze_block in freeze_list:
    for name, param in freeze_block.named_parameters():
        param.requires_grad = False
    freeze_block = freeze_block.eval()

### NICE Validation

In [ ]:
def denormalize_image(normalized_image, mean, std):
    image = normalized_image.numpy().transpose(1, 2, 0)
    image = std * image + mean
    image = np.clip(image, 0, 1)
    
    return image

In [ ]:
from tqdm import tqdm

def inference(dataloader, model, processor):
    results = {
        'public_id': [],
        'caption': [],
    }

    for inputs, filenames in tqdm(dataloader):
        inputs.to(start_device, dtype)
        generated_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)
        generated_texts = [text.strip() for text in processor.batch_decode(generated_ids, skip_special_tokens=True)]
        results['public_id'] += filenames
        results['caption'] += generated_texts
        
    return results

In [ ]:
import os
import csv

def save_dict_to_csv(dict_to_save, save_path):
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    rows = list(map(list, zip(*dict_to_save.values())))
    with open(save_path, 'w') as f:
        w = csv.writer(f)
        w.writerow(dict_to_save.keys())
        w.writerows(rows)

In [ ]:
from dataset_config import nice_dataset_config
from torch.utils.data import DataLoader
from custom_datasets.nice_dataset import NICETestDataset
from datasets import Dataset

## NICEValDataset으로 바꾸고 평가 지표 알 수 있도록 하면 좋을듯
nice_val_ds = NICETestDataset(
    nice_dataset_config['val_image_folder'],
    processor.image_processor,
)

nice_val_dataloader = DataLoader(nice_val_ds, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True)

In [ ]:
generated_texts = inference(nice_val_dataloader, model, processor)

In [ ]:
save_dict_to_csv(generated_texts, result_save_path)